# Remote deployment of a Cloud AI Platform Pipeline

This example requires that you have a Cloud AI Platform Pipelines installation up and running.
The easiest way to run this example locally is to have `gcloud` [installed](https://cloud.google.com/sdk/install) and [authorized for your account](https://cloud.google.com/sdk/docs/initializing).

You will also need to have `kfp` installed in your (virtual) environment:
```sh
pip install kfp
```

First, do some imports:

In [ ]:
import datetime

import kfp
import kfp.compiler as compiler
import kfp.dsl as dsl

Define a (very) simple example pipeline to run:

In [ ]:
@dsl.pipeline(
  name='Sequential',
  description='A pipeline with two sequential steps.'
)
def sequential_pipeline(filename='gs://ml-pipeline-playground/shakespeare1.txt'):
  """A pipeline with two sequential steps."""

  op1 = dsl.ContainerOp(
     name='getfilename',
     image='library/bash:4.4.23',
     command=['sh', '-c'],
     arguments=['echo "%s" > /tmp/results.txt' % filename],
     file_outputs={'newfile': '/tmp/results.txt'})
  op2 = dsl.ContainerOp(
     name='echo',
     image='library/bash:4.4.23',
     command=['sh', '-c'],
     arguments=['echo "%s"' % op1.outputs['newfile']]
     )

Next we'll create an instance of the Kubeflow Pipelines client. For this you need the endpoint of your installation. An easy way to do this is to visit your AI Pipelines dashboard, and click on  **SETTINGS**.  
![Click 'Settings' to get information about your installation](https://storage.googleapis.com/amy-jo/images/kfp-deploy/FOubezufx6b.png)

A window will pop up that looks similar to the following:
![KFP client settings](https://storage.googleapis.com/amy-jo/images/kfp-deploy/9SD6WeYgpTv-2.png)

Copy and paste the code snippet to replace the code below with your endpoint:

In [ ]:
import kfp
# TODO: edit the following for your installation endpoint
client = kfp.Client(host='<your-endpoint>.pipelines.googleusercontent.com')

List your installation's existing pipelines to sanity-check the connection.  If you haven't yet uploaded any of your own pipelines, you'll just see the example ones listed.

In [ ]:
client.list_pipelines()

Compile the pipeline, and create a Pipelines `Experiment`:

In [ ]:
compiler.Compiler().compile(sequential_pipeline, '/tmp/sequential.tar.gz')
exp = client.create_experiment(name='sequential')

Finally, run the pipeline:

In [ ]:
ts = int(datetime.datetime.utcnow().timestamp() * 100000)
res =  client.run_pipeline(exp.id, 'sequential_' + str(ts), 
                           '/tmp/sequential.tar.gz',
                          )
print(res)

------------------------------------
Copyright 2020, Google, LLC.
Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.